In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.15.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
xy = np.loadtxt('/content/drive/MyDrive/dataset/example_data/logistic_regression_dataset_16_features (1).csv', delimiter=',', dtype=np.float32)
x_train = xy[0:-100, 0:-1]
y_train = xy[0:-100, [-1]]

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(900, 16) (900, 1)
(100, 16) (100, 1)


In [ ]:
print(x_test)
print(y_test)

[[-0.7244851   1.3556042  -0.7149444  ... -0.85890603 -1.9053533
  -0.2046343 ]
 [-0.8029174   0.6253252  -0.04345221 ...  2.240588    1.76353
  -1.7704611 ]
 [-0.02266894  1.6755017   0.02397674 ...  1.4200407  -1.5515019
  -0.6292267 ]
 ...
 [-0.03057244  0.9291818  -0.5378848  ...  1.4716104  -0.01332215
   1.3443856 ]
 [ 1.5770882   0.22941801  0.39344442 ... -1.7842149  -0.14629813
   0.23497753]
 [-0.8128021   0.41440588  0.28651828 ...  1.1447431   0.05587194
   0.6620453 ]]
[[0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


In [ ]:
W = tf.Variable(tf.random.normal([16, 1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')


## 가설 설정
* 병이 있다 / 없다로 분류
* binary classification으로 진행

## $$
\begin{align}
sigmoid(x) & = \frac{1}{1+e^{-x}}  \\\\\
\end{align}
$$

In [ ]:
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(-(tf.matmul(features, W) + b)))
    return hypothesis

print(logistic_regression(x_train))

tf.Tensor(
[[9.85380113e-01]
 [5.73623598e-01]
 [9.84757900e-01]
 [9.67658222e-01]
 [1.50122810e-02]
 [4.26204056e-01]
 [8.71432796e-02]
 [9.20974135e-01]
 [9.69867408e-01]
 [1.55124650e-03]
 [9.29989934e-01]
 [2.21748045e-03]
 [7.17376471e-01]
 [3.94767194e-05]
 [4.24985588e-01]
 [9.55845475e-01]
 [1.31493673e-01]
 [2.10923306e-03]
 [5.11143029e-01]
 [1.66259687e-02]
 [2.35374493e-04]
 [8.47961903e-01]
 [2.38673575e-02]
 [4.34233516e-01]
 [8.73126626e-01]
 [9.23128307e-01]
 [9.93501484e-01]
 [1.39849052e-01]
 [3.70965570e-01]
 [3.09542775e-01]
 [8.81050885e-01]
 [9.87172842e-01]
 [7.86203325e-01]
 [6.95159554e-01]
 [1.40788689e-01]
 [9.99084592e-01]
 [3.22147012e-02]
 [9.86716866e-01]
 [1.28051132e-01]
 [9.90217865e-01]
 [7.91971564e-01]
 [4.56335358e-02]
 [2.17242613e-01]
 [4.81626153e-01]
 [1.53575242e-02]
 [6.53201044e-01]
 [4.26781356e-01]
 [6.74717128e-01]
 [2.04445302e-01]
 [9.98741806e-01]
 [3.20103243e-02]
 [9.15305674e-01]
 [9.76229787e-01]
 [3.35602999e-01]
 [6.64264802e-03]

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [ ]:
def loss_fn(hypothesis, labels):
    loss = -tf.reduce_mean(labels * tf.math.log(hypothesis) + \
                           (1 - labels) * tf.math.log(1 - hypothesis))
    return loss

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.005)

## 학습

In [ ]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
      pred = logistic_regression(features)
      loss_value = loss_fn(pred, labels)
      grads = tape.gradient(loss_value, [W,b])
      optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
      if step % 100 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

Iter: 0, Loss: 0.7933
Iter: 100, Loss: 0.7348
Iter: 200, Loss: 0.6819
Iter: 300, Loss: 0.6341
Iter: 400, Loss: 0.5912
Iter: 500, Loss: 0.5527
Iter: 600, Loss: 0.5183
Iter: 700, Loss: 0.4875
Iter: 800, Loss: 0.4601
Iter: 900, Loss: 0.4356
Iter: 1000, Loss: 0.4138
Iter: 1100, Loss: 0.3944
Iter: 1200, Loss: 0.3770
Iter: 1300, Loss: 0.3614
Iter: 1400, Loss: 0.3475
Iter: 1500, Loss: 0.3351
Iter: 1600, Loss: 0.3239
Iter: 1700, Loss: 0.3139
Iter: 1800, Loss: 0.3049
Iter: 1900, Loss: 0.2969
Iter: 2000, Loss: 0.2896
Iter: 2100, Loss: 0.2830
Iter: 2200, Loss: 0.2771
Iter: 2300, Loss: 0.2718
Iter: 2400, Loss: 0.2670
Iter: 2500, Loss: 0.2626
Iter: 2600, Loss: 0.2587
Iter: 2700, Loss: 0.2551
Iter: 2800, Loss: 0.2518
Iter: 2900, Loss: 0.2489
Iter: 3000, Loss: 0.2462
Iter: 3100, Loss: 0.2437
Iter: 3200, Loss: 0.2414
Iter: 3300, Loss: 0.2394
Iter: 3400, Loss: 0.2374
Iter: 3500, Loss: 0.2357
Iter: 3600, Loss: 0.2341
Iter: 3700, Loss: 0.2325
Iter: 3800, Loss: 0.2311
Iter: 3900, Loss: 0.2298
Iter: 4000, 

## 테스트

In [ ]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [ ]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.8800
